In [3]:
import os

import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.ndimage.filters import gaussian_filter
from pathlib import Path
import json
from IPython.display import display
from PIL import Image

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys.ecephys_session import (
    EcephysSession, 
    removed_unused_stimulus_presentation_columns
)
from allensdk.brain_observatory.ecephys.visualization import plot_mean_waveforms, plot_spike_counts, raster_plot
from allensdk.brain_observatory.visualization import plot_running_speed

# tell pandas to show all columns when we display a DataFrame
pd.set_option("display.max_columns", None)

import warnings
warnings.filterwarnings('ignore')

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19808\1149438173.py:8: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter


## Set the path where data is stored

In [4]:
# Example cache directory path, it determines where downloaded data will be stored
output_dir = r'D:\Neuroscience'

# this path determines where downloaded data will be stored
manifest_path = os.path.join(output_dir, "manifest.json")

cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

print(cache.get_all_session_types())

sessions = cache.get_session_table()
brain_observatory_type_sessions = sessions[sessions["session_type"] == "brain_observatory_1.1"]

['brain_observatory_1.1', 'functional_connectivity']


In [11]:
brain_observatory_sessions = np.array(brain_observatory_type_sessions.index)

In [12]:
# 因为每个刺激长为250ms，所以定义函数把每个刺激分成25个时间单元
def calculate_time_unit(row):
    time = row['time_since_stimulus_presentation_onset']
    if time >= 0.24:
        return 25
    else:
        # 将时间转换为对应的单位，每0.01秒为一个单位，即使时间采样率为100Hz，从1开始编号
        return np.floor(time / 0.01) + 1

对每一个stimulus_presentation_id，基于time_since_stimulus列，如果time_since_stimulus <= 0.01，则time unit为1，如果0.01 < time_since_stimulus <=0.02，则time unit为2，....，如果time_since_stimulus >= 0.24，则time unit为25。这样做有一个前提，每个stimulus都必须有0.24s之后的刺激，因为如果没有的话，此次stimulus就只有24个time unit而不是25个。

In [13]:
def calculate_times(session_id):
    # 获取session表
    session = cache.get_session_data(session_id)
    ns_presentation_ids = session.stimulus_presentations.loc[
        (session.stimulus_presentations['stimulus_name'] == 'natural_scenes')
    ].index.values
    
    # 获取初始times表(列不全，需要添加其他信息)
    times = session.presentationwise_spike_times(
        stimulus_presentation_ids=ns_presentation_ids,
    )
    
    # 获取units表
    units = session.units
    
    # 该类型刺激开始的时间点：times.index[0] - times['time_since_stimulus_presentation_onset'].loc[times.index[0]]
    # 计算每一行所属的block(每个session的natural scense分为3个block)
    times['spike_time'] = times.index
    times['spike_time'] = times['spike_time'].astype('float64')
    times['time_diff'] = times['spike_time'].diff()
    times['is_new_block'] = (times['time_diff'] > 1).astype(int)
    times['natural_scenes_block_id'] = times['is_new_block'].cumsum()
    times.drop(['time_diff', 'is_new_block'], axis = 1, inplace = True)

    flag = 0
    for i in times.groupby('stimulus_presentation_id')['time_since_stimulus_presentation_onset'].max():
        if i < 0.24:
            flag = 1
            print('对于session {}, 存在stimulus在刺激末尾没有unit产生神经反应, 在对应stimulus末尾的time_unit加上全0行以补齐。'.format(session_id))
    if flag == 0:
        print('对于session {}, 每个stimulus在每个time_unit内均有神经反应。'.format(session_id))
    
    # 应用函数创建新列
    times['time_unit'] = times.apply(calculate_time_unit, axis=1).astype(int)

    # 时间补齐
    for index, item in times.groupby('stimulus_presentation_id')['time_unit'].max().iteritems():
        if item < 25:
            for time_unit in range(item + 1, 26):
                new_row = {'stimulus_presentation_id': index, 'time_unit': time_unit}
                times = times.append(new_row, ignore_index = True)
    
    # 计算每个stimulus_presentation_id的time_unit最大值累加偏移量
    offsets = times.groupby('stimulus_presentation_id')['time_unit'].max().cumsum().shift(1).fillna(0)
    
    # 将偏移量应用到每个time_unit上
    times['adjusted_time_unit'] = times['time_unit'] + times['stimulus_presentation_id'].map(offsets)
    times['adjusted_time_unit'] = times['adjusted_time_unit'].astype(int)
    
    # 从session.unit表中引入cluster_id，使每一列代表一个神经元。要用left_join，因为没有unit激活的时间单元自然不会有cluster_id，但希望保留这一行
    units = session.units
    times = pd.merge(times, units[['cluster_id', 'structure_acronym']], 
                    left_on = 'unit_id', right_on = units.index, how = 'left').sort_values(by = 'time_unit')

    # 按列排序
    times = times.sort_values(by = ['stimulus_presentation_id', 'adjusted_time_unit'], ascending = [True, True])

    return times



In [34]:
def calculate_df_annotation(sessions, times, session_id, start_neuron_num):
    
    # 数据透视表得到每一个time_unit，每一个神经元的激活次数
    df = pd.pivot_table(times, index = ['stimulus_presentation_id', 'adjusted_time_unit'], columns = ['cluster_id'], aggfunc = 'size')
    df = df.fillna(0).astype(int)

    # 取出原来的第一级索引和第二级索引
    level_1_index = df.index.get_level_values(0)
    level_2_index = df.index.get_level_values(1)
    
    # 创建完整的第二级索引范围
    full_level_2_range = np.arange(1, 148751)
    
    # 找出缺失的第二级索引, 如果有缺失值，进入补齐过程
    missing_level_2 = np.setdiff1d(full_level_2_range, level_2_index)
    if len(missing_level_2) != 0:
        
        # 初始化一个空的DataFrame，用于存放缺失的行
        missing_rows = []
        for missing in missing_level_2:
            # 找到缺失索引之前的第一级索引
            prev_index = np.where(level_2_index < missing)[0]
        
            if len(prev_index) > 0:
                prev_level_1_index = level_1_index[prev_index[-1]]
            else:
                prev_level_1_index = np.nan  # 若没有前一个索引值，则设置为 NaN
            # 创建一个新行，使用prev_level_1_index作为第一级索引，missing作为第二级索引，值全为0
            missing_rows.append((prev_level_1_index, missing))
        # 创建一个新的DataFrame包含这些缺失的行
        missing_df = pd.DataFrame(0, index=pd.MultiIndex.from_tuples(missing_rows, names=('level_1', 'level_2')), columns=df.columns)
        # 将原来的DataFrame和缺失的DataFrame合并
        df = pd.concat([df, missing_df]).sort_index()
    

    # 获取session表
    session = cache.get_session_data(session_id)
    ns_presentation_ids = session.stimulus_presentations.loc[
        (session.stimulus_presentations['stimulus_name'] == 'natural_scenes')
    ].index.values
    
    # 获取units表
    units = session.units
    
    # 建立annotation file记录元数据
    annotation = units[['cluster_id', 'structure_acronym']].drop_duplicates()
    annotation['session'] = session_id
    annotation = annotation.sort_values(by = 'cluster_id')
    
    # 设置一个字典，给每一个神经元起名字
    sorted_cluster_id = np.sort(annotation['cluster_id'].unique())
    dict_cluster_id = {}
    for i in range(len(sorted_cluster_id)):
        dict_cluster_id[sorted_cluster_id[i]] = 'neuron' + '_' + str(i + start_neuron_num)
    
    # 给annotation file的神经元起别名
    annotation['cluster_id'] = annotation['cluster_id'].replace(dict_cluster_id)
    
    # 给df的神经元起别名
    df.rename(columns = dict_cluster_id, inplace = True)

    annotation['specimen_id'] = sessions['specimen_id'].loc[session_id]
    annotation['session_type'] = sessions['session_type'].loc[session_id]
    annotation['age_in_days'] = sessions['age_in_days'].loc[session_id]
    annotation['sex'] = sessions['sex'].loc[session_id]
    annotation['full_genotype'] = sessions['full_genotype'].loc[session_id]

    # 记录最大的neuron number，防止不同session的neuron重叠
    max_neuron_num = len(sorted_cluster_id) + start_neuron_num - 1
    
    return df, annotation, max_neuron_num



In [35]:
# 去掉session 737581020因为只有5912个natural scenes stimulus
brain_observatory_sessions = np.array(brain_observatory_sessions.tolist()[:4] + brain_observatory_sessions.tolist()[5:])

In [36]:
brain_observatory_sessions

array([715093703, 719161530, 721123822, 732592105, 739448407, 742951821,
       743475441, 744228101, 746083955, 750332458, 750749662, 751348571,
       754312389, 754829445, 755434585, 756029989, 757216464, 757970808,
       758798717, 759883607, 760345702, 760693773, 761418226, 762120172,
       762602078, 763673393, 773418906, 791319847, 797828357, 798911424,
       799864342])

In [37]:
# 起始编号为0，之后每次编号为前一个session最大编号+1
start_neuron_num = 0
for session_id in brain_observatory_sessions:
    times = calculate_times(session_id)
    df, annotation, max_neuron_num = calculate_df_annotation(sessions, times, session_id, start_neuron_num)
    df.to_csv(r'D:\Neuroscience\neuropixel_data\stimulus_response(individual_sessions)\session_{}.csv'.format(session_id))
    annotation.to_csv(r'D:\Neuroscience\neuropixel_data\annotation\annotation_{}.csv'.format(session_id))
    start_neuron_num = max_neuron_num + 1

对于session 715093703, 每个stimulus在每个time_unit内均有神经反应。
对于session 719161530, 每个stimulus在每个time_unit内均有神经反应。
对于session 721123822, 每个stimulus在每个time_unit内均有神经反应。
对于session 732592105, 每个stimulus在每个time_unit内均有神经反应。
对于session 739448407, 存在stimulus在刺激末尾没有unit产生神经反应, 在对应stimulus末尾的time_unit加上全0行以补齐。
对于session 739448407, 存在stimulus在刺激末尾没有unit产生神经反应, 在对应stimulus末尾的time_unit加上全0行以补齐。
对于session 739448407, 存在stimulus在刺激末尾没有unit产生神经反应, 在对应stimulus末尾的time_unit加上全0行以补齐。
对于session 739448407, 存在stimulus在刺激末尾没有unit产生神经反应, 在对应stimulus末尾的time_unit加上全0行以补齐。
对于session 739448407, 存在stimulus在刺激末尾没有unit产生神经反应, 在对应stimulus末尾的time_unit加上全0行以补齐。
对于session 739448407, 存在stimulus在刺激末尾没有unit产生神经反应, 在对应stimulus末尾的time_unit加上全0行以补齐。
对于session 742951821, 每个stimulus在每个time_unit内均有神经反应。
对于session 743475441, 每个stimulus在每个time_unit内均有神经反应。
对于session 744228101, 每个stimulus在每个time_unit内均有神经反应。
对于session 746083955, 每个stimulus在每个time_unit内均有神经反应。
对于session 750332458, 每个stimulus在每个time_unit内均有神经反应。
对于session 750749662, 每个stimulus在每个

In [2]:
# 定义包含 CSV 文件的文件夹路径
folder_path = r'D:\Neuroscience\neuropixel_data\stimulus_response(individual_sessions)'

# 获取文件夹下所有 CSV 文件的文件名
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
dataframes = []
# 读取每个 CSV 文件并添加到列表中
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path, dtype='int32')  # 使用较低内存的 dtype
    dataframes.append(df)

# 横向合并所有 DataFrame
combined_df = pd.concat(dataframes, axis=1)


In [3]:
combined_df.to_csv(r'D:\Neuroscience\neuropixel_data\stimulus_response(individual_sessions)\all_sessions.csv')

In [4]:
# 定义包含 CSV 文件的文件夹路径
folder_path = r'D:\Neuroscience\neuropixel_data\annotation'

# 获取文件夹下所有 CSV 文件的文件名
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
dataframes = []
# 读取每个 CSV 文件并添加到列表中
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path) 
    dataframes.append(df)

# 纵向合并所有 DataFrame
combined_annotation = pd.concat(dataframes, axis=0)

In [5]:
combined_annotation.to_csv(r'D:\Neuroscience\neuropixel_data\annotation\all_annotations.csv')

In [18]:
# # 确定每个session包含多少natural scenes stimulus
# session_stimulus_num = []
# for session_id in brain_observatory_sessions:
#     session = cache.get_session_data(session_id)
#     ns_presentation_ids = session.stimulus_presentations.loc[
#         (session.stimulus_presentations['stimulus_name'] == 'natural_scenes')
#     ].index.values
#     times = session.presentationwise_spike_times(
#     stimulus_presentation_ids=ns_presentation_ids,
#     )
#     stimulus_num = len(times.groupby('stimulus_presentation_id').count())
#     session_stimulus_num.append(stimulus_num)
# session_stimulus_num

[5950,
 5950,
 5950,
 5950,
 5912,
 5950,
 5950,
 5950,
 5950,
 5950,
 5950,
 5950,
 5950,
 5950,
 5950,
 5950,
 5950,
 5950,
 5950,
 5950,
 5950,
 5950,
 5950,
 5950,
 5950,
 5950,
 5950,
 5950,
 5950,
 5950,
 5950,
 5950]